# Linear Regression

*In my opinion, Linear Regression is the most important Machine Learning model that exists. It is worth to study well.*

* predicting continous variables is commonplace in data science
* the results are interpretable
* can solve complex problems with proper engineering of features
* Linear Regression has been backed up by deep statistical research
* Linear Regression is the base to understand other linear models (such as Logistic Regression or Poisson Regression)
* Prerequisite for understanding neural networks
* very often does the job sufficiently well

In [ ]:
import pickle

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

### 1. Define Business Goal

**We predict prices of houses from a multitude of features.**

A detailed description of the challenge can be found on [Kaggle.com](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

### 2. Get Data

In [ ]:
df = pd.read_csv('../data/housing/housing_train.csv', index_col=0)
df.head(3)

### 3. Split Data into Training, Validation and Test sets
The test set has been already sliced off by Kaggle. They only give us the input data but not the correct results.

In [ ]:
train, val = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train.shape, val.shape

### 4. Explore Data

In [ ]:
train['SalePrice'].hist(bins=20)

In [ ]:
COLUMNS = ['LotFrontage', 'OverallQual', 'YearBuilt']
train[COLUMNS].isna().sum().plot.barh()

In [ ]:
sns.pairplot(train[COLUMNS + ['SalePrice']])

### 4. Define X and y

In [ ]:
# X is a matrix of input features
Xtrain = train[COLUMNS]
Xval = val[COLUMNS]

# y is a vector of scalar values --> Regression
ytrain = train['SalePrice']
yval = val['SalePrice']

In [ ]:
Xtrain.shape, ytrain.shape

In [ ]:
Xval.shape, yval.shape

### 5. Feature Engineering

In [ ]:
# patch up the missing data
avg = Xtrain.mean()
Xtrain = Xtrain.fillna(avg)
Xval = Xval.fillna(avg)

### 6. Train a Linear Regression Model

The model is:

$\hat y = w_1x_1 + w_2x_2 + .. + w_nx_n + w_0$

There are two ways to solve a Linear Regression Model

#### a) Normal Equation

Uses an analytical approach to calculate coefficients directly.
This is a closed-form solution called the **Normal Equation**

The Normal Equation has two big disadvantages:

* quadratic time complexity $O(N^2)$
* it can gets stuck if your features are redundant

Usually, b) is the better choice.

#### b) Gradient Descent

Iteratively optimizes the coefficients to find the lowest possible MSE.

* always finds the minimum (MSE is a convex function)
* partial derivative (linear time complexity to data points and features)

This is the implementation used in practically all common libraries (scikit, statsmodels, R, Spark, TensorFlow).

In [ ]:
m = LinearRegression(fit_intercept=True)
m.fit(Xtrain, ytrain)
m.score(Xtrain, ytrain)  # R^2 value

### 7. Evaluate the Model

#### R squared

* 0 = no explainability from the model's correlation
* 1 = the model completely explains the variance in the data

#### MSE

* Mean Squared Error
* tt is very sensitve to outliers - each residual is squared, so
* residuals greater than one have a disproportionate big effect on outliers
* residuals less than one have a disproportionate small effect on outliers

#### MAE

* Mean Absolute Error
* average of the absolute residuals
* less sensitive to outliers than the MSE
* same unit as the target variable

#### RMSL

* Root Mean Squared Log Error
* doesn't penalise over-estimates as much as underestimates
* good for count data that stretches over several orders of magnitude


In [ ]:
ypred = m.predict(Xtrain)
mse_train = mean_squared_error(ytrain, ypred)
mae_train = mean_absolute_error(ytrain, ypred)

print(f"training MSE {mse_train:4.2f}")
print(f"training MAE {mae_train:4.2f}")

In [ ]:
ypred_val = m.predict(Xval)
mse_val = mean_squared_error(yval, ypred_val)
mae_val = mean_absolute_error(yval, ypred_val)
print(f"validation MSE {mse_val:4.2f}")
print(f"validation MAE {mae_val:4.2f}")

In [ ]:
# inspect the coefficients

# LotFrontage, OverallQual, YearBuilt
m.coef_.round(1), m.intercept_.round(1)

### 8. Deploy the model
here we just save the model to a file to use it elsewhere.

In [ ]:
pickle.dump(m, open('../models/linreg.pkl', 'wb'))

### 9. Prediction for new data

In [ ]:
# LotFrontage, OverallQual, YearBuilt
house = [[50, 10, 1975]]
m.predict(house)

### 10. Model with Statsmodels

**Statsmodels** adds more interpretability to the model. The interface does not fit 100% to scikit though.

In [ ]:
from statsmodels.regression.linear_model import OLS

In [ ]:
Xtrain['intercept'] = 1  # <-- OLS does not do this on its own

In [ ]:
sm = OLS(ytrain, Xtrain)  # opposite order
result = sm.fit()
result.summary()

## Assumptions of Linear Regression

Linear Regression describes a relationship between two or more variables that tries to find a hyperplane through the data that minimizes the MSE.

We model an output variable y as a linear combination of input variables X.

**Prediction:**

$\hat{y} = \hat{w}X$ or

$\hat{y} = \hat{w_1} * x_1 + \hat{w_2} * x_2 + ... + \hat{w_n} * x_n + \hat{w_0}$


**True Relationship:**

$y = Xw + \epsilon$

where

- X ($x_1, x_2, x_3, ..., x_n$) are the input features, e.g. size of the house, quality, year it was built
- $\hat{y}$ is our prediction for the outcome value (cnt; count of rentals)
- $\hat{w}$ $(w_0, w_1, ..., w_n)$ are the estimated coefficients for our input features
- $\epsilon$ is an error term (some unexplainable randomness)

## 4 + 3 + 1 Assumptions

There are several assumptions that you will want to check before you use a LinReg model in production.

* **The first 4 assumptions guarantee that the best possible MSE is found** (by the Gauss-Markov Theorem). If they hold, you can use the model to make predictions.
* **The next 3 assumptions provide a basis for explainability**. If they hold, you can interpret the p-values of coefficients.
* **The last assumption (autocorrelation) is important for time series data.**

### A1) The true relationship between X and y is linear in the parameters

The dependent variable, y, is linearly depending on the independent variable(s), x, and the error, $\epsilon$ as 

$y = w_0 + w_1 * x + \epsilon$

To check this assumption, look at the scatterplot of the data.

### A2) The sample is a random sample

The training data has to be representative for the whole population, otherwise the model **fails to generalize**.
This assumption mainly depends on your data collection/cleaning. It is difficult to check at this point.

### A3) There is variation in the X variables

The X variable(s) take(s) on different values. Otherwise no information can be gained by looking at X.

To check this assumption, plot the histograms of your features.

### A4) Zero Conditional Mean

The mean of the error term $\epsilon$ conditional on X is 0.
There is no relationship between X and the error term $\epsilon$.

$E(\epsilon|X) = 0$

To check this assumption, plot the **residuals**:

In [ ]:
sns.residplot(Xtrain['LotFrontage'], ytrain)

In [ ]:
sns.residplot(Xtrain['YearBuilt'], ytrain)

**If these assumptions hold, the model is unbiased**

----

### A5) Homoskedasticity

Homoskedasticity means that the variance of the y does not change over time. 

#### Check #1: Plot the residuals against X

In [ ]:
residuals = ypred - ytrain
plt.plot(Xtrain['YearBuilt'], residuals, 'k.')

#### Check #2: Breusch-Pagan Test

What the Breusch-Pagan-Test does intuitively:

- Runs a linear regression of y on X
- It calculates the residuals
- It runs a linear regression of the residuals on X $\hat{\epsilon} = \delta_0 + \delta_1 * x_1$
- It uses the $R^2$ value to determine whether the explanatory variables are able to explain the residuals; If that is the case, then we have some kind of heteroscedasticity

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

In [ ]:
het_breuschpagan(residuals, Xtrain)
# The fourth value is important: it gives you the p-value
# The Null hypothesis is that there is no heteroskedasticity
# In this case we reject the null hypothesis because the p-value is very small

### A6) Normal Distribution of the Error Terms

This assumption is convenient if we want to make precise statements about effect sizes (the w coefficients of the linear regression).

Let's look at statsmodels output again.

In [ ]:
residuals.hist(bins=20)

In [ ]:
# better: QQ-plot
from scipy.stats import probplot

probplot(residuals, plot=plt)

In [ ]:
# Jarque-Bera-Test
from scipy.stats import jarque_bera

jarque_bera(residuals)
# second value is the p-value
# if we fail to reject the Null Hypothesis (p > 0.05)
# we can confidently say that the residuals are probably normally distributed

What if normality fails to hold?

If the sample size is big and we have many features, this is not a problem. 
We can use the Central Limit Theorem and still apply the statistical evaluation.

### A7) No Multicolinearity

If this assumption fails, we still don't have any problems to make predictions. Statements about the coefficients become very difficult if we have multicolinearity.

What is Multicolinearity?<br>
Two features are perfectly colinear, if one is a linear transformation of the other. For example height in cm and height in m would be perfectly colinear.

There will always be correlation between different variables. So when do we consider it to be a problem.


In [ ]:
# Calculate Variance inflation factors
from statsmodels.stats.outliers_influence import variance_inflation_factor

Xt = add_constant(Xtrain)
pd.Series([variance_inflation_factor(Xtrain.values, i)
           for i in range(Xtrain.shape[1])],
          index=Xtrain.columns)

# A VIF greater than 5 indicates high multicolinearity: